In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import json
from CocktailEmbeddingMaker import CocktailEmbeddingMaker
from CocktailEmbeddingMaker import Eval
# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)
with open('../category.json', 'r') as f:
    category_data = json.load(f)
# with open('user_list_v1_5.json', 'r') as f:
#     test_user_list = json.load(f)

attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
#가상의 유저 만들기
test_user_list = []
user_num = 5
for i in range(user_num):
    user = {}
    # user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[1:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)

#음료의 총 량
total_amount = 200
#모델 로드
#D:\CODE\feelflask_master\backend\data_works\bidirectional_lstm.h5
# load_model = keras.models.load_model('bidirectional_lstm.h5')
load_model = keras.models.load_model('best_model_earthy.h5')
#Eval 객체 생성 -> inference용 전용 class가 필요하면 이야기할것
Eval_obj = Eval(json_data,flavor_data,category_data,load_model)
list_result_2 = []
# 알콜 : 12개
# 믹서: 7개 
# 부재료:8개 
limited_ingredient_list= [  'peach schnapps','baileys irish cream','kahlua','triple sec','malibu rum','tequila',
                            'whisky','jack daniels','malibu rum','midori melon liqueur','vodka','light rum',
                            "cranberry juice","lime juice",'lemon juice',"orange juice","tonic water", "milk",'sugar syrup',
                            'powdered sugar','salt','sugar','ice','cinnamon','black pepper','grenadine','carbonated water'
                            ]
user_num=5
for i in range(user_num):
    # print(f"[OUTPUT]user_recipe's profile : {json.dumps(user_recipe_profile,indent=4)}")
    user_seed = Eval_obj.select_user_seed(test_user_list[i])
    generated_recipes = Eval_obj.generate_recipe(load_model,user_seed,test_user_list[i])
    result_recipe ={}

    for recipe, ingredients in zip(generated_recipes[0], generated_recipes[1]):
        result_recipe[recipe]=ingredients * total_amount
    user_recipe_profile = Eval_obj.get_taste_log(generated_recipes)

    print(" please ignore user_id!")
    print(f"[INPUT] user preference: {json.dumps(test_user_list[i],indent=4)}")
    print(f"[OUTPUT]recipe!!!!: {result_recipe} ")
    list_result_2.append(result_recipe)

#Limited Gen
    new_ingredient_list = []
    
    Eval_obj.set_limited_ingredient(limited_ingredient_list)
    user_seed = Eval_obj.select_user_seed(test_user_list[i])
    generated_recipes_2 = Eval_obj.generate_recipe_limited_withModel(load_model,user_seed,test_user_list[i])
    print(f"[OUTPUT]generated_recipes_2!!!!: {generated_recipes_2} ")
    #  recipe_ingredients, available_ingredients,user_preference)
    best_ingredient = Eval_obj.find_similar_ingredients(generated_recipes[0],limited_ingredient_list,test_user_list[i])
    target_abv = test_user_list[i]['ABV']
    quantities = Eval_obj.adjust_ingredient_quantities(best_ingredient, target_abv, test_user_list[i])
    # new_ingredient_list.append(best_ingredient)
    print(f"[OUTPUT]best_ingredient: {best_ingredient}, quantities : {quantities}")
    


user_seed : wine
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
 please ignore user_id!
[INPUT] user preference: {
    "ABV": 3,
    "boozy": 67,
    "sweet": 51,
    "sour": 66,
    "bitter": 90,
    "umami": 58,
    "salty": 42,
    "astringent": 18,
    "Perceived_temperature": 79,
    "spicy": 92,
    "herbal": 77,
    "floral": 26,
    "fruity": 36,
    "nutty": 96,
    "creamy": 35,
    "smoky": 48
}
[OUTPUT]recipe!!!!: {'wine': 42.82662943407086, 'powdered sugar': 39.29334264148228, 'lemon': 39.29334264148228, 'egg white': 39.29334264148228, 'cherry': 39.29334264148228} 
user_seed : baileys irish cream
seed_ingredient : baileys irish cream
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[OUTPUT]generated_recipes_2!!!!: (['baileys irish cream', 'kahlua', 'grenadine', 'milk'], [0.10084705866189972, 0.11857605264

In [2]:
best_ingredient

{'cranberry juice', 'tequila', 'triple sec'}

In [6]:
a = next((item for item in Eval_obj.flavor_data if item["name"] == 'whisky'), None)
a

{'name': 'whisky',
 'ABV': 40.0,
 'boozy': 70.0,
 'sweet': 10.0,
 'sour': 0.0,
 'bitter': 20.0,
 'umami': 20.0,
 'salty': 0.0,
 'astringent': 0.0,
 'Perceived_temperature': 30.0,
 'spicy': 0.0,
 'herbal': 0.0,
 'floral': 0.0,
 'fruity': 10.0,
 'nutty': 0.0,
 'creamy': 0.0,
 'smoky': 20.0,
 'ID': 294}

In [3]:
a = ['a','b','c']
b = ['d','e','f']
a+b

['a', 'b', 'c', 'd', 'e', 'f']

In [ ]:
a = Eval_obj.map_ingredient('aaa')
alcohol_list = [ingredient for ingredient in user_seed_list if ingredient in Eval_obj.limited_ingredient_list and Eval_obj.get_ingredient_category(ingredient) == 'Alcohol']

In [ ]:
category_data

# 카테고리별로 재료를 저장할 딕셔너리 초기화
categorized_ingredients = {}

# 각 재료를 반복하면서 카테고리별로 분류
for ingredient, categories in category_data.items():
    for category in categories:
        if category not in categorized_ingredients:
            categorized_ingredients[category] = []
        categorized_ingredients[category].append(ingredient)

# 결과 출력
for category, ingredients in categorized_ingredients.items():
    print(f"Category: {category}")
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"{ingredient}")
    print()

In [ ]:
import json
with open('../category.json', 'r') as f:
    category_data = json.load(f)
# JSON 문자열을 파싱하여 딕셔너리로 변환
# 카테고리별로 재료를 저장할 딕셔너리 초기화
categorized_ingredients = {}

# 각 재료를 반복하면서 카테고리별로 분류
for ingredient, categories in category_data.items():
    for category in categories:
        if category not in categorized_ingredients:
            categorized_ingredients[category] = []
        categorized_ingredients[category].append(ingredient)

# 결과 출력
for category, ingredients in categorized_ingredients.items():
    print(f"Category: {category}")
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"- {ingredient}")
    print()

In [ ]:
category_data